In [1]:
# Notebook 3 - basic processing of the data, joining files 
# together and removing unwanted sites.

In [2]:
import pandas as pd     

In [3]:
########################################################
# The EA publish daily data on https://environment.data.gov.uk/
# flood-monitoring/archive giving river levels about once every 
# couple of hours.  Each file is a huge dataset, so my first 
# task will be to extract the data from each of the stations 
# I am interested in, ditch the rest and put them together into 
# one dataframe lasting multiple days (possibly a month 
# eventually?)
# I will aim to start with the last 5 days of February
########################################################

In [4]:
# The file has "stationReference" as the key field I can search 
# on to tie back to the data I have, without searching on a url.
# From previously:
#            ~ Cowley Bridge  = 45117
#            ~ Upton Hellions = 45136
#            ~ Yeoford        = 45137

In [5]:
########################################################
#
# Taking individual day files, and combining them into big whole
# month df taking 3 sites of interest our of the bigdf saving 
# the whole month file back to my data folder with a new name
#
########################################################

In [6]:
# Taking files and joining them into one df.  
# Note I've been through this and manually changed the date range
# I am interested in each time, rather than retype the code.
# For only me this was just quicker than turning it into a 
# function on this occasion.
                                                             
startdate = "2024-03-01"     # start for subset of data
enddate = "2024-03-31"       # end of the month I've downloaded
startdt = pd.to_datetime(startdate, format='%Y-%m-%d')        
enddt = pd.to_datetime(enddate, format='%Y-%m-%d')

date_range = pd.date_range(start=startdt, end=enddt, freq='D')

print(date_range)

dayfmt = '%Y-%m-%d' 
mylist = []
for today in date_range:
    mylist.append(pd.read_csv(f"data/readings-full-{today.strftime(dayfmt)}.csv"))
bigdf = pd.concat(mylist)

bigdf["dateTime"] = pd.to_datetime(bigdf["dateTime"], 
                                   format='%Y-%m-%dT%H:%M:%SZ')
bigdf.info()

DatetimeIndex(['2024-03-01', '2024-03-02', '2024-03-03', '2024-03-04',
               '2024-03-05', '2024-03-06', '2024-03-07', '2024-03-08',
               '2024-03-09', '2024-03-10', '2024-03-11', '2024-03-12',
               '2024-03-13', '2024-03-14', '2024-03-15', '2024-03-16',
               '2024-03-17', '2024-03-18', '2024-03-19', '2024-03-20',
               '2024-03-21', '2024-03-22', '2024-03-23', '2024-03-24',
               '2024-03-25', '2024-03-26', '2024-03-27', '2024-03-28',
               '2024-03-29', '2024-03-30', '2024-03-31'],
              dtype='datetime64[ns]', freq='D')


C:\Users\suzan\AppData\Local\Temp\ipykernel_13420\3776385295.py:19: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  mylist.append(pd.read_csv(f"data/readings-full-{today.strftime(dayfmt)}.csv"))
C:\Users\suzan\AppData\Local\Temp\ipykernel_13420\3776385295.py:19: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  mylist.append(pd.read_csv(f"data/readings-full-{today.strftime(dayfmt)}.csv"))
C:\Users\suzan\AppData\Local\Temp\ipykernel_13420\3776385295.py:19: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  mylist.append(pd.read_csv(f"data/readings-full-{today.strftime(dayfmt)}.csv"))
C:\Users\suzan\AppData\Local\Temp\ipykernel_13420\3776385295.py:19: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  mylist.append(pd.read_csv(f"data/readings-full-{today.strftime(dayfmt)}.csv"))


<class 'pandas.core.frame.DataFrame'>
Index: 12324808 entries, 0 to 89291
Data columns (total 13 columns):
 #   Column            Dtype         
---  ------            -----         
 0   dateTime          datetime64[ns]
 1   date              object        
 2   measure           object        
 3   station           object        
 4   label             object        
 5   stationReference  object        
 6   parameter         object        
 7   qualifier         object        
 8   datumType         object        
 9   period            int64         
 10  unitName          object        
 11  valueType         object        
 12  value             object        
dtypes: datetime64[ns](1), int64(1), object(11)
memory usage: 1.3+ GB


In [7]:
# so having extracted all data to one big dataframe, I want to 
# get out just the data for the sites I am interested in
mask_for_cowley = bigdf["stationReference"] == "45117"
dfcowley = bigdf[mask_for_cowley]
mask_for_upton = bigdf["stationReference"] == "45136"
dfupton = bigdf[mask_for_upton]
mask_for_yeoford = bigdf["stationReference"] == "45137"
dfyeoford = bigdf[mask_for_yeoford]
df = pd.concat([dfcowley, dfupton, dfyeoford], axis = 0)
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 6964 entries, 11383 to 66186
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   dateTime          6964 non-null   datetime64[ns]
 1   date              6964 non-null   object        
 2   measure           6964 non-null   object        
 3   station           6964 non-null   object        
 4   label             6964 non-null   object        
 5   stationReference  6964 non-null   object        
 6   parameter         6964 non-null   object        
 7   qualifier         6964 non-null   object        
 8   datumType         0 non-null      object        
 9   period            6964 non-null   int64         
 10  unitName          6964 non-null   object        
 11  valueType         6964 non-null   object        
 12  value             6964 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(11)
memory usage: 761.7+ KB


In [8]:
# outputting data to new .csv file with just the data I want in
# note I've manually changed the date in here each month to save 
# the new file.  I am aware that I could download concat dates 
# in one go, but for the sake of my laptop I've chosen to do it
# a month at a time, knowing that my bigdf is pretty big!

df.to_csv("data/df-2024-03.csv", encoding='utf-8', index=False)

In [9]:
df.head() # simply confirming the data is as I want it to be.

,dateTime,date,measure,station,label,stationReference,parameter,qualifier,datumType,period,unitName,valueType,value
11383,2024-03-01 00:45:00,2024-03-01,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Cowley Bridge,45117,level,Stage,NaN,900,m,instantaneous,0.808
14330,2024-03-01 00:15:00,2024-03-01,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Cowley Bridge,45117,level,Stage,NaN,900,m,instantaneous,0.809
14355,2024-03-01 00:30:00,2024-03-01,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Cowley Bridge,45117,level,Stage,NaN,900,m,instantaneous,0.806
14380,2024-03-01 00:00:00,2024-03-01,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Cowley Bridge,45117,level,Stage,NaN,900,m,instantaneous,0.809
26832,2024-03-01 01:45:00,2024-03-01,http://environment.data.gov.uk/flood-monitorin...,http://environment.data.gov.uk/flood-monitorin...,Cowley Bridge,45117,level,Stage,NaN,900,m,instantaneous,0.803


In [10]:
######################################################
#
# Taking each month file and bolting them together into one 
# large file for longer duration saving multi month file back 
# with new file name 
#
######################################################

In [11]:
# Having combined single day files into month files and then 
# extracted the sites I wanted, for each month, I need to then
# combine these files into a year of data I am interested in.

In [12]:
# Taking files of just my sites and joining them into one df.  
                                                             
startdate = "2023-03"        # start for subset of data            
enddate = "2024-03"          # end of the month I've downloaded
startdt = pd.to_datetime(startdate, format='%Y-%m')        
enddt = pd.to_datetime(enddate, format='%Y-%m')

date_range = pd.date_range(start=startdt, end=enddt, freq='M')

print(date_range)

dayfmt = '%Y-%m' 
mylist = []
for month in date_range:
    mylist.append(pd.read_csv(f"data/df-{month.strftime(dayfmt)}.csv"))
df_all = pd.concat(mylist)


df_all.info()

DatetimeIndex(['2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30',
               '2023-07-31', '2023-08-31', '2023-09-30', '2023-10-31',
               '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29'],
              dtype='datetime64[ns]', freq='M')
<class 'pandas.core.frame.DataFrame'>
Index: 88314 entries, 0 to 4802
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   dateTime          88314 non-null  object 
 1   date              88314 non-null  object 
 2   measure           88314 non-null  object 
 3   station           88314 non-null  object 
 4   label             88314 non-null  object 
 5   stationReference  88314 non-null  int64  
 6   parameter         88314 non-null  object 
 7   qualifier         88314 non-null  object 
 8   datumType         0 non-null      float64
 9   period            88314 non-null  float64
 10  unitName          88314 non-null  object 
 11  valueType         8

In [13]:
# outputting data to new .csv file with just the data I want in
df_all.to_csv("data/df_all.csv", encoding='utf-8', index=False)

In [14]:
# So df_all now contains 13 months of data.